In [1]:
import pickle
import os
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
def build_predictions(audio_dir):
    y_true = []
    y_pred = []
    fn_prob = {}
    
    print('Extracting features from audio')
    for fn in tqdm(os.listdir(audio_dir)):
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        label = fn2class[fn]
        c = classes.index(label)
        y_prob = []
        
        for i in range(0,wav.shape[0]-config.step, config.step):
            sample = wav[i:i+config.step]
            X =  mfcc(sample, rate,numcep=config.nfeat, 
                      nfilt=config.nfilt, nfft=config.nfft)
            X = (X - config.min)/(config.max - config.min)
            
            if config.mode == 'conv':
                X = X.reshape(1, X.shape[0], X.shape[1], 1)
            elif config.mode == 'time':
                X = np.expand_dims(X, axis=0)
            elif config.mode == 'mlp':
                X = np.expand_dims(X, axis=0)
            y_hat = model.predict(X)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
            
        fn_prob[fn] = np.mean(y_prob, axis=0).flatten()
    
    return y_true, y_pred, fn_prob

In [3]:
df = pd.read_csv('C:/Users/hp/Desktop/Audio-Classification-v5/train_sound.csv')
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname, df.label))
p_path = os.path.join('pickles','mlp.p')

with open(p_path, 'rb') as handle:
    config = pickle.load(handle)
    
model = load_model(config.model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
y_true, y_pred, fn_prob = build_predictions('C:/Users/hp/Desktop/Audio-Classification-v5/train_files')  #interchangeable wavfiles/clean
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)

Extracting features from audio


100%|██████████████████████████████████████| 2700/2700 [03:50<00:00, 12.84it/s]


In [5]:
print(acc_score)

0.8268724279835391


In [21]:
y_probs = []

for i, row in df.iterrows():
    y_prob = fn_prob[row.fname]
    y_probs.append(y_prob)
    for c, p in zip(classes, y_prob):
        df.at[i, c] = p

y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred 

df.to_csv('mlp_prediction.csv', index=False)